获得训练数据

In [9]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
# 要导入代码的路径 ,utils无法导入的同学,添加上自己code的路径 ,项目代码结构 code/utils ....
sys.path.append('../')  # 返回notebook的上一级目录
# sys.path.append('E:\GitHub\QA-abstract-and-reasoning')  # 效果同上

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
from utils.config import *
from utils.loader import *

In [78]:
# 词向量的载入
wv_model = word2vec.Word2Vec.load(WV_MODEL)
vocab_index, index_vocab = load_vocab(VOCAB_INDEX)
embedding_matrix = np.loadtxt(EMBEDDING_MATRIX)  # 

In [36]:
# 预处理数据载入
train_seg = pd.read_csv(TRAIN_SEG).fillna("")
test_seg = pd.read_csv(TEST_SEG).fillna("")
print(train_seg.shape,"\n",test_seg.shape)
train_seg.head()

(82943, 6) 
 (20000, 5)


,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先 挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,JEEP,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 C180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场价 钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


In [39]:
train_seg['X'] = train_seg[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
test_seg['X'] = test_seg[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
train_seg['X'].head()

0    方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重...
1    奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 有发 一下 发动...
2    2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...
3    30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 ...
4    2012 款 奔驰 C180 维修保养 动力 值得 拥有 家庭 用车 入手 维修保养 费用 ...
Name: X, dtype: object

## 句子长度不一样: 填充

In [55]:
def pad(sentence, max_len, vocab):
    """
    <START> <END> <PAD> <UNK>
    """

    # 0.按空格统计切分出词
    words = sentence.strip().split(' ')
    # 1. [截取]规定长度的词数
    words = words[:max_len]
    # 2. 填充< unk > ,判断是否在vocab中, 不在填充 < unk >
    sentence = [word if word in vocab else '<UNK>' for word in words]
    # 3. 填充< start > < end >
    sentence = ['<START>'] + sentence + ['<STOP>']
    # 4. 判断长度，填充　< pad >
    sentence = sentence + ['<PAD>'] * (max_len - len(words))
    return ' '.join(sentence)

In [49]:
train_seg['X'][0]

'方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问题 跑 快 还好 点 倒车 重 很 非常 重 累人 觉得 车主 以前 没 重选 助理 泵 换 不行 放 机换 现在 还 不 知道 车主 解释'

In [56]:
pad(train_seg['X'][0], 60, vocab_index)

'<START> 方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问题 跑 快 还好 点 倒车 重 很 非常 重 累人 觉得 车主 以前 没 <UNK> 助理 泵 换 不行 放 机换 现在 还 不 知道 车主 解释 <STOP> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

## 获取适当的max_len

最大长度值 = 每句话长度均值 + 2倍每句话长度标准差

In [58]:
def get_max_len(data):
    """
    获得合适的最大长度值
    :param data: 待统计的数据  train_df['Question']
    :return: 最大长度值
    """
    max_lens = data.apply(lambda x: x.count(' '))
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [66]:
# 获取输入数据 适当的最大长度
train_x_max_len = get_max_len(train_seg['X'])
test_x_max_len = get_max_len(test_seg['X'])

x_max_len = max(train_x_max_len, test_x_max_len)

# 获取标签数据 适当的最大长度
train_y_max_len = get_max_len(train_seg['Report'])

In [69]:
print("训练集x最大长度 {}".format(train_x_max_len))
print("测试集x最大长度 {}".format(test_x_max_len))
print("训练集y最大长度 {}".format(train_y_max_len))

训练集x最大长度 247
测试集x最大长度 258
训练集y最大长度 31


In [70]:
# 训练集X处理
train_seg['X'] = train_seg['X'].apply(lambda x: pad(x, x_max_len, vocab_index))
# 训练集Y处理
train_seg['Y'] = train_seg['Report'].apply(lambda x: pad(x, train_y_max_len, vocab_index))
# 测试集X处理
test_seg['X'] = test_seg['X'].apply(lambda x: pad(x, x_max_len, vocab_index))

In [75]:
# 保存中间结果数据
train_seg['X'].to_csv(TRAIN_X_PAD, index=None, header=False)
train_seg['Y'].to_csv(TRAIN_Y_PAD, index=None, header=False)
test_seg['X'].to_csv(TEST_X_PAD, index=None, header=False)

## 更新词表

In [80]:
print('start retrain w2v model')
wv_model.build_vocab(LineSentence(TRAIN_X_PAD), update=True)
wv_model.train(LineSentence(TRAIN_X_PAD), epochs=5, total_examples=wv_model.corpus_count)
print('1/3')
wv_model.build_vocab(LineSentence(TRAIN_Y_PAD), update=True)
wv_model.train(LineSentence(TRAIN_Y_PAD), epochs=5, total_examples=wv_model.corpus_count)
print('2/3')
wv_model.build_vocab(LineSentence(TEST_X_PAD), update=True)
wv_model.train(LineSentence(TEST_X_PAD), epochs=5, total_examples=wv_model.corpus_count)

start retrain w2v model
1/3
2/3


(8690390, 26000000)

In [83]:
# 保存词向量模型
wv_model.save(WV_MODEL_PAD)

In [96]:
embedding_matrix.shape, wv_model.wv.vectors.shape

((32562, 300), (32566, 300))

In [98]:
# 更新vocab和embedding
vocab_index = {word: index for index, word in enumerate(wv_model.wv.index2word)}
index_vocab = {index: word for index, word in enumerate(wv_model.wv.index2word)}
embedding_matrix = wv_model.wv.vectors

In [100]:
train_seg['X'].head()

0    <START> 方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向...
1    <START> 奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 ...
2    <START> 2010 款 宝马X1 2011 年 出厂 20 排量 通用 <UNK> 变...
3    <START> 30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 ...
4    <START> 2012 款 奔驰 C180 维修保养 动力 值得 拥有 家庭 用车 入手 ...
Name: X, dtype: object

In [102]:
# 遇到未知词就填充unk的索引
unk_index = vocab_index['<UNK>']
def transform_data(sentence,vocab):
    # 字符串切分成词
    words=sentence.split(' ')
    # 按照vocab的index进行转换
    ids=[vocab[word] if word in vocab else unk_index for word in words]
    return ids

In [109]:
train_seg['X'][0][:40]

'<START> 方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向'

In [107]:
transform_data(train_seg['X'][0],vocab_index)[:10]

[32562, 403, 985, 247, 231, 403, 4, 10, 94, 4]

In [111]:
# 将词转换成索引  [<START> 方向机 重 ...] -> [32800, 403, 986, 246, 231
train_ids_x=train_seg['X'].apply(lambda x:transform_data(x,vocab_index))
train_ids_y=train_seg['Y'].apply(lambda x:transform_data(x,vocab_index))
test_ids_x=test_seg['X'].apply(lambda x:transform_data(x,vocab_index))

In [113]:
train_ids_x.head()

0    [32562, 403, 985, 247, 231, 403, 4, 10, 94, 4,...
1    [32562, 816, 26474, 388, 219, 361, 1610, 33, 2...
2    [32562, 1445, 81, 5125, 1290, 64, 1138, 375, 6...
3    [32562, 10244, 9, 307, 62, 523, 170, 933, 331,...
4    [32562, 1348, 81, 816, 8963, 3282, 208, 2079, ...
Name: X, dtype: object